In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('../mtr-datasets/atp7d.arff')
data = pd.DataFrame(data)
data.head()

,"""'departwb'""","""'departdaytype'""","""'fetchdays'""","""'fetchdow'""","""'dtd'""","""'ALLminpA'""","""'ALLminpA'l1""","""'ALLminpA'l2""","""'ALLminpA'l7""","""'ALLavgpA'""",...,"""fdisoweekday4""","""fdisoweekday5""","""fdisoweekday6""","""fdisoweekday7""",LBL+ALLminpA+bt7d_000,LBL+ALLminp0+bt7d_000,LBL+aDLminpA+bt7d_000,LBL+aCOminpA+bt7d_000,LBL+aFLminpA+bt7d_000,LBL+aUAminpA+bt7d_000
0,72.0,1.0,60.0,3.0,12.0,359.0,372.0,351.0,317.0,615.96,...,0.0,0.0,0.0,0.0,326.0,675.0,626.0,545.0,326.0,545.0
1,79.0,1.0,69.0,5.0,10.0,336.0,298.0,298.0,320.0,616.73,...,0.0,1.0,0.0,0.0,315.0,675.0,631.0,587.0,315.0,587.0
2,79.0,1.0,70.0,6.0,9.0,336.0,336.0,298.0,320.0,628.87,...,0.0,0.0,1.0,0.0,315.0,675.0,631.0,587.0,315.0,587.0
3,79.0,1.0,71.0,7.0,8.0,336.0,336.0,336.0,330.0,632.73,...,0.0,0.0,0.0,1.0,315.0,675.0,631.0,587.0,315.0,587.0
4,79.0,1.0,72.0,1.0,7.0,336.0,336.0,336.0,330.0,635.47,...,0.0,0.0,0.0,0.0,315.0,1182.0,720.0,720.0,315.0,720.0


In [3]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values
#Making Scaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

#Train-Test Split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=45)

In [4]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (207, 411)
Train_Y:  (207, 6)
Test_X:  (89, 411)
Test_Y:  (89, 6)


In [5]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))


## KNN Regression

In [11]:
from sklearn.neighbors import KNeighborsRegressor

In [12]:
for i in range(1,20):
    model = KNeighborsRegressor(n_neighbors=i)
    model.fit(X_train,y_train)
    print(i,":",model.score(X_test,y_test))    

1 : 0.23725207072160084
2 : 0.5330462712562838
3 : 0.6660494405588339
4 : 0.63044677066312
5 : 0.6222600801321555
6 : 0.6392982336882812
7 : 0.6366710647635537
8 : 0.6184016833573813
9 : 0.6086543390494291
10 : 0.6060998481353256
11 : 0.6045599699989668
12 : 0.5807846219038963
13 : 0.5654395100770074
14 : 0.5580459453521959
15 : 0.5494047289512743
16 : 0.5329274359896564
17 : 0.5212322135381551
18 : 0.5111269036972198
19 : 0.49835596013076994


In [13]:
model = KNeighborsRegressor(n_neighbors=9)
model.fit(X_train,y_train)
evaluate(y_scaler.inverse_transform(y_test),y_scaler.inverse_transform(model.predict(X_test)))

RMSE:  72.99318156165046
MAE:  38.86787349146899
R_Squared:  0.6086543390494291


## Random Forest

In [28]:
from sklearn.ensemble import RandomForestRegressor

In [29]:
model = RandomForestRegressor(n_estimators=5)
model.fit(X_train,y_train)
evaluate(y_scaler.inverse_transform(y_test),y_scaler.inverse_transform(model.predict(X_test)))

RMSE:  67.78970511700274
MAE:  36.37865168539326
R_Squared:  0.6421566694153461


## ANN 

In [30]:
from keras.models import Sequential
from keras.layers import Dense

In [31]:
model = Sequential()
model.add(Dense(128,activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(64,activation='relu'))
model.add(Dense(6,activation='linear'))
model.compile(optimizer="adam",loss='mse')

In [32]:
model.fit(X_train,y_train,epochs=500,batch_size=50)

Epoch 1/500
5/5 [==============================] - 0s 1ms/step - loss: 0.9197
Epoch 2/500
5/5 [==============================] - 0s 1ms/step - loss: 0.5678
Epoch 3/500
5/5 [==============================] - 0s 1ms/step - loss: 0.4029
Epoch 4/500
5/5 [==============================] - 0s 1ms/step - loss: 0.3275
Epoch 5/500
5/5 [==============================] - 0s 1ms/step - loss: 0.2960
Epoch 6/500
5/5 [==============================] - 0s 1ms/step - loss: 0.2282
Epoch 7/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1914
Epoch 8/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1608
Epoch 9/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1408
Epoch 10/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1318
Epoch 11/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1327
Epoch 12/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1229
Epoch 13/500
5/5 [==============================] - 0s 1ms/st

5/5 [==============================] - 0s 1ms/step - loss: 0.0143
Epoch 105/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0124
Epoch 106/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0150
Epoch 107/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0123
Epoch 108/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0096
Epoch 109/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0079
Epoch 110/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0081
Epoch 111/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0130
Epoch 112/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0100
Epoch 113/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0068
Epoch 114/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0105
Epoch 115/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0049
Epoch 116/500
5/5 [==============================] - 0

5/5 [==============================] - 0s 1ms/step - loss: 0.0034
Epoch 204/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0051
Epoch 205/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0040
Epoch 206/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0033
Epoch 207/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0026
Epoch 208/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0021
Epoch 209/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0021
Epoch 210/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 211/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0044
Epoch 212/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0051
Epoch 213/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0043
Epoch 214/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0034
Epoch 215/500
5/5 [==============================] - 0

5/5 [==============================] - 0s 1ms/step - loss: 1.7367e-04
Epoch 305/500
5/5 [==============================] - 0s 1ms/step - loss: 1.6939e-04
Epoch 306/500
5/5 [==============================] - 0s 2ms/step - loss: 1.3441e-04
Epoch 307/500
5/5 [==============================] - 0s 1ms/step - loss: 1.6719e-04
Epoch 308/500
5/5 [==============================] - 0s 1ms/step - loss: 1.5501e-04
Epoch 309/500
5/5 [==============================] - 0s 2ms/step - loss: 2.5695e-04
Epoch 310/500
5/5 [==============================] - 0s 1ms/step - loss: 1.9709e-04
Epoch 311/500
5/5 [==============================] - 0s 2ms/step - loss: 1.4267e-04
Epoch 312/500
5/5 [==============================] - 0s 1ms/step - loss: 1.8521e-04
Epoch 313/500
5/5 [==============================] - 0s 1ms/step - loss: 1.6454e-04
Epoch 314/500
5/5 [==============================] - 0s 1ms/step - loss: 1.4033e-04
Epoch 315/500
5/5 [==============================] - 0s 1ms/step - loss: 2.4342e-04
Epoch 

5/5 [==============================] - 0s 1ms/step - loss: 2.4058e-04
Epoch 402/500
5/5 [==============================] - 0s 1ms/step - loss: 3.5953e-04
Epoch 403/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0016
Epoch 404/500
5/5 [==============================] - 0s 1ms/step - loss: 9.7449e-04
Epoch 405/500
5/5 [==============================] - 0s 1ms/step - loss: 6.7903e-04
Epoch 406/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 407/500
5/5 [==============================] - 0s 2ms/step - loss: 9.1024e-04
Epoch 408/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0032
Epoch 409/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0034
Epoch 410/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0031
Epoch 411/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0047
Epoch 412/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0041
Epoch 413/500
5/5 [===============

In [33]:
evaluate(y_scaler.inverse_transform(y_test),y_scaler.inverse_transform(model.predict(X_test)))

RMSE:  59.4488794396582
MAE:  35.11453349938553
R_Squared:  0.6978647907343478
